In [7]:
import os

In [8]:
!pwd

/f/python_venv/End-to-End-Cancer-Classification-using-MLFlow-DVC/research


In [9]:
os.chdir("../")
!pwd

/f/python_venv/End-to-End-Cancer-Classification-using-MLFlow-DVC


In [10]:
# create an entity - a class created with a dataclass decorator.
# a class in which we can define any class variables without using self.
# when we set frozen=True, we are stating that we do not allow adding any other variables inside that class.
# by creating an entity, we can create a custom return type for a function.

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
# configuration manager in src/CNN_Classifier/config

from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
            )
    
        return data_ingestion_config
        

In [ ]:
# component in : src/CNN_Classifier/components

import os
import zipfile
import gdown
from CNN_Classifier import logger
from CNN_Classifier.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self) -> str:
        '''
        Fetch data from the url
        '''

        try:
            dataset_url = self.config.source_URL
            zip_dataset_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Doanloading data from {dataset_url} into file {zip_dataset_dir}.")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(url=prefix+file_id, output=zip_dataset_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_dataset_dir}.")

        except Exception as e:
            raise e

    def extract_zip_file(self):
        '''
        reads config.zip_file_path: str
        Extracts the zip file into the data directory.
        Function returns None
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)




In [23]:
# pipeline: step-by-step calling method

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-04 14:19:54,536] [INFO] [common.read_yaml:33] [yaml file: config\config.yaml loaded successfully]
[2025-08-04 14:19:54,536] [INFO] [common.read_yaml:33] [yaml file: params.yaml loaded successfully]
[2025-08-04 14:19:54,541] [INFO] [common.create_directories:52] [Created directory at: artifacts]
[2025-08-04 14:19:54,542] [INFO] [common.create_directories:52] [Created directory at: artifacts/data_ingestion]
[2025-08-04 14:19:54,544] [INFO] [689623926.download_data:20] [Doanloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view into file artifacts/data_ingestion/Chest-CT-Scan-data.zip.]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=8d4b814e-38af-4d59-adec-b55dd384c45c
To: f:\python_venv\End-to-End-Cancer-Classification-using-MLFlow-DVC\artifacts\data_ingestion\Chest-CT-Scan-data.zip
100%|██████████| 49.0M/49.0M [00:01<00:00, 31.1MB/s]

[2025-08-04 14:19:59,724] [INFO] [689623926.download_data:26] [Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view into file artifacts/data_ingestion/Chest-CT-Scan-data.zip.]
